In [1]:
%config InlineBackend.rc = {"figure.dpi": 72, "figure.figsize": (6.0, 4.0)}
%matplotlib ipympl

import abtem
import ase
import matplotlib.pyplot as plt
import numpy as np
from ase.cluster import Decahedron

In [2]:
substrate = ase.build.bulk("C", cubic=True)

substrate *= (12, 12, 12)

bl = 1.54  # Bond length
rng = np.random.default_rng(seed=10)
substrate.positions[:] += rng.normal(size=(len(substrate), 3)) * 0.5 * bl

substrate.wrap()

In [3]:
p = 6
q = 2
r = 1

nanoparticle = Decahedron("Au", p, q, r, latticeconstant=None)
nanoparticle.rotate(30, "x", center="cop")

nanoparticle.center(vacuum=1)

nanoparticle.set_cell(substrate.cell)
nanoparticle.center(axis=(0, 1))
nanoparticle.translate((0, 0, substrate.cell[2, 2] - 2))

atoms = nanoparticle + substrate

atoms.center(axis=2, vacuum=2)

In [4]:
wave = abtem.PlaneWave(energy=200e3)

potential = abtem.Potential(atoms, sampling=0.1, exit_planes=4)

exit_wave = wave.multislice(potential).compute().ensure_lazy()

image = exit_wave.apply_ctf(
    defocus=np.linspace(0, 100, 10),
    Cs=np.linspace(0, 1e6, 10),
    semiangle_cutoff=np.linspace(10, 50, 10),
)

[########################################] | 100% Completed | 2.68 ss


In [9]:
image[:,:,:,1:].compute().show(interact=True)

[########################################] | 100% Completed | 43.11 s


KeyboardInterrupt: 